# Explore verejne zakazky
elastic search should be running for this

In [1]:
from elasticsearch import Elasticsearch
es = Elasticsearch()

In [2]:
index = 'verejnezakazky_2018-02-13'
type_vz = 'verejnazakazka'
type_pz = 'profilzadavatele'
docs_vz = {'index': index, 'doc_type': type_vz}
docs_pz = {'index': index, 'doc_type': type_pz}

## Verejne zakazky

In [3]:
print("Total count:")
vz_total = es.count(**docs_vz)['count']
vz_total

Total count:


185790

In [6]:
ds = es.search(**docs_vz)['hits']['hits']
ds[0]

{'_id': '8700CC9E1FF5D48FD7F29DC100B35B85',
 '_index': 'verejnezakazky_2018-02-13',
 '_score': 1.0,
 '_source': {'cPV': ['16000000-5'],
  'dataset': 'VVZ-2006',
  'datumUverejneni': '2014-09-09T00:00:00',
  'displayId': 0,
  'dodavatele': [],
  'dokumenty': [],
  'evidencniCisloZakazky': '496393',
  'formulare': [{'cislo': '7452010096393',
    'druh': 'F52',
    'onlyOnProfile': False,
    'uRL': '',
    'zverejnen': '2014-09-09T00:00:00'}],
  'id': '8700CC9E1FF5D48FD7F29DC100B35B85',
  'kriteria': [],
  'lastUpdated': '0001-01-01T00:00:00',
  'posledniZmena': '2014-09-09T00:00:00',
  'zadavatel': {'iCO': '62698044', 'jmeno': 'Pavel Francl'}},
 '_type': 'verejnazakazka'}

In [56]:
def n_unique_approx(docs_param, field):
    """
    This is elastic native way how to count distinct v
    """
    body = {'size': 0,
            'aggs': {'distinct_ids': {'cardinality': {'field': field}}}}
    return es.search(**docs_param, body=body)['aggregations']['distinct_ids']['value']

In [57]:
def n_unique_exact(docs_param, field):
    page = es.search(
        **docs_param,
        scroll='2m',
        size=1000,
        body={"query": {"match_all": {}}})
    sid = page['_scroll_id']
    scroll_size = page['hits']['total']

    # Start scrolling
    elems = set()
    while(scroll_size > 0):
        page = es.scroll(scroll_id=sid, scroll='2m')
        sid = page['_scroll_id']
        # Get the number of results that we returned in the last scroll
        scroll_size = len(page['hits']['hits'])

        elems.update(
            {entry['_source'][field] for entry in page['hits']['hits']})
    
    return len(elems)

Is `evidencniCisloZakazky` and `id` unique? 

In [59]:
def print_n_unique(field):
    print(f"{field}: {n_unique_exact(docs_vz, field)}/{vz_total}")
print_n_unique('evidencniCisloZakazky')
print_n_unique('id') 
print_n_unique('displayId')

evidencniCisloZakazky: approx. 179209/185790
id: approx. 184790/185790
displayId: approx. 1/185790


In [23]:
body={"query": {"term": {"dataset": "VVZ-2016"}}}
es.search(**docs_vz, body=body)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'B27CAE54F33ED60E3CE947ABCE495C96',
    '_index': 'verejnezakazky_2018-02-13',
    '_score': 1.8918245,
    '_source': {'cPV': [],
     'dataset': 'VVZ-2016',
     'datumUverejneni': '2016-12-23T00:00:00+01:00',
     'displayId': 0,
     'dodavatele': [],
     'dokumenty': [],
     'evidencniCisloZakazky': 'Z2016-007561',
     'formulare': [{'cislo': 'F2016-007561',
       'druh': 'CZ05',
       'onlyOnProfile': False,
       'uRL': '',
       'zverejnen': '2016-12-23T00:00:00'}],
     'id': 'B27CAE54F33ED60E3CE947ABCE495C96',
     'kriteria': [],
     'lastUpdated': '2016-12-23T00:00:00',
     'posledniZmena': '2016-12-23T00:00:00',
     'zadavatel': {'iCO': '60153245',
      'jmeno': 'Gymnázium, Vrchlabí, Komenského 586'}},
    '_type': 'verejnazakazka'},
   {'_id': '1A946F6DCF60D530617B639E8375744A',
    '_index': 'verejnezakazky_2018-02-13',
    '_score': 1.8918245,
    '_source': {'cPV

## Profily zadavatele

In [14]:
print("Total count:")
es.count(**docs_pz)['count']

Total count:


18859

In [15]:
ds = es.search(**docs_pz)['hits']['hits']
ds[0]

{'_id': 'VVZ-2016-403861',
 '_index': 'verejnezakazky_2018-02-13',
 '_score': 1.0,
 '_source': {'dataSet': 'VVZ-2016',
  'datumUverejneni': '2014-12-09T00:00:00+01:00',
  'evidencniCisloFormulare': 'F2014-552076',
  'evidencniCisloProfilu': '403861',
  'id': 'VVZ-2016-403861',
  'lastAccess': '2018-02-08T19:31:56.7769608+01:00',
  'lastAccessResult': 0,
  'url': 'http://enjoy-trade.profilzadavatele.cz/',
  'zadavatel': {'iCO': '26950707', 'jmeno': 'Enjoy Trade, s.r.o.'}},
 '_type': 'profilzadavatele'}